![LetNet-4](image\LetNet-4.jpg)

In [1]:
#LetNet4 

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO) #作用：将 TensorFlow 日志信息输出到屏幕

In [2]:
#定义模型函数
def LetNet4_model_fn(features, labels, mode):
  """Model function for LetNet4."""
  # Input Layer
  # Reshape X to 4-D tensor: [batch_size, width, height, channels]
  # MNIST images are 28x28 pixels, and have one color channel
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  # Computes 6 features using a 5x5 filter with ReLU activation.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 28, 28, 1]
  # Output Tensor Shape: [batch_size, 28, 28, 6]
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=6,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  # First max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 28, 28, 6]
  # Output Tensor Shape: [batch_size, 14, 14, 6]
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2
  # Computes 16 features using a 5x5 filter.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 14, 14, 6]
  # Output Tensor Shape: [batch_size, 14, 14, 16]
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=16,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #2
  # Second max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 14, 14, 16]
  # Output Tensor Shape: [batch_size, 7, 7, 16]
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Flatten tensor into a batch of vectors
  # Input Tensor Shape: [batch_size, 7, 7, 16]
  # Output Tensor Shape: [batch_size, 7 * 7 * 16]
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 16])

  # Dense Layer
  # Densely connected layer with 84 neurons
  # Input Tensor Shape: [batch_size, 7 * 7 * 16]
  # Output Tensor Shape: [batch_size, 84]
  dense = tf.layers.dense(inputs=pool2_flat, units=84, activation=tf.nn.relu)

  # Add dropout operation; 0.6 probability that element will be kept
  #dropout = tf.layers.dropout(
  #    inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits layer
  # Input Tensor Shape: [batch_size, 84]
  # Output Tensor Shape: [batch_size, 10]
  logits = tf.layers.dense(inputs=dense, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  # 每500步一个Iteration
  if mode == tf.estimator.ModeKeys.TRAIN:
    boundaries = [500*2, 500*5, 500*8, 500*12]
    values = [0.0005, 0.0002, 0.0001, 0.00005, 0.00001]
    global_step = tf.train.get_or_create_global_step()
    learning_rate = tf.train.piecewise_constant(global_step, boundaries, values)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


In [3]:
#定义estimator
def main(unused_argv):
  # Load training and eval data
  mnist = tf.contrib.learn.datasets.load_dataset("mnist")
  train_data = mnist.train.images  # Returns np.array
  train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
  eval_data = mnist.test.images  # Returns np.array
  eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

  # Create the Estimator
  logdir = "model/mnist_LetNet4_model"
  count = "2"
  LetNet4_classifier = tf.estimator.Estimator(
      model_fn=LetNet4_model_fn, model_dir=logdir + count)

  # Set up logging for predictions
  # Log the values in the "Softmax" tensor with label "probabilities"
  #tensors_to_log = {"probabilities": "softmax_tensor"}
  #logging_hook = tf.train.LoggingTensorHook(
  #    tensors=tensors_to_log, every_n_iter=500)
  
  # 运行10个Iterations
  for i in range(20):
      # Train the model
      train_input_fn = tf.estimator.inputs.numpy_input_fn(
          x={"x": train_data},
          y=train_labels,
          batch_size=100,
          num_epochs=None,
          shuffle=True)
      LetNet4_classifier.train(
          input_fn=train_input_fn,
          steps=600) #可加参数hooks=[logging_hook]

      # Evaluate the model and print results 
      eval_input_fn = tf.estimator.inputs.numpy_input_fn(
          x={"x": eval_data}, y=eval_labels, num_epochs=1, shuffle=False)
      eval_results = LetNet4_classifier.evaluate(input_fn=eval_input_fn)
      print(eval_results)


In [ ]:
#运行程序
if __name__ == "__main__":
  tf.app.run()

Instructions for updating:
Please use tf.data.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'model/mnist_LetNet4_model2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_

INFO:tensorflow:loss = 0.35564968, step = 1001
INFO:tensorflow:global_step/sec: 12.5598
INFO:tensorflow:loss = 0.35637608, step = 1101 (7.964 sec)
INFO:tensorflow:global_step/sec: 12.7264
INFO:tensorflow:loss = 0.26582935, step = 1201 (7.856 sec)
INFO:tensorflow:global_step/sec: 12.8782
INFO:tensorflow:loss = 0.2628868, step = 1301 (7.765 sec)
INFO:tensorflow:global_step/sec: 12.8047
INFO:tensorflow:loss = 0.2731928, step = 1401 (7.940 sec)
INFO:tensorflow:Saving checkpoints for 1500 into model/mnist_LetNet4_model2\model.ckpt.
INFO:tensorflow:Loss for final step: 0.25075272.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-05-07:45:25
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/mnist_LetNet4_model2\model.ckpt-1500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-03-05-07:45:29
INFO:tensorflow:Saving

INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 3500 into model/mnist_LetNet4_model2\model.ckpt.
INFO:tensorflow:loss = 0.30506533, step = 3501
INFO:tensorflow:global_step/sec: 11.5531
INFO:tensorflow:loss = 0.3071965, step = 3601 (8.656 sec)
INFO:tensorflow:global_step/sec: 9.89957
INFO:tensorflow:loss = 0.2631931, step = 3701 (10.101 sec)
INFO:tensorflow:global_step/sec: 11.9948
INFO:tensorflow:loss = 0.15096909, step = 3801 (8.337 sec)
INFO:tensorflow:global_step/sec: 10.5267
INFO:tensorflow:loss = 0.19559479, step = 3901 (9.500 sec)
INFO:tensorflow:Saving checkpoints for 4000 into model/mnist_LetNet4_model2\model.ckpt.
INFO:tensorflow:Loss for final step: 0.41267502.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-05-07:49:18
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/mnist_LetNet4_model2\model.ckpt-4000
INFO:tensorflow:Running local_i

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/mnist_LetNet4_model2\model.ckpt-6000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 6000 into model/mnist_LetNet4_model2\model.ckpt.
INFO:tensorflow:loss = 0.34039962, step = 6001
INFO:tensorflow:global_step/sec: 11.553
INFO:tensorflow:loss = 0.26409864, step = 6101 (8.659 sec)
INFO:tensorflow:global_step/sec: 11.5661
INFO:tensorflow:loss = 0.2239017, step = 6201 (8.645 sec)
INFO:tensorflow:global_step/sec: 11.2697
INFO:tensorflow:loss = 0.34460184, step = 6301 (8.881 sec)
INFO:tensorflow:global_step/sec: 11.1851
INFO:tensorflow:loss = 0.360626, step = 6401 (8.940 sec)
INFO:tensorflow:Saving checkpoints for 6500 into model/mnist_LetNet4_model2\model.ckpt.
INFO:tensorflow:Loss for final step: 0.2579455.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-05-07:53

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/mnist_LetNet4_model2\model.ckpt-8500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 8500 into model/mnist_LetNet4_model2\model.ckpt.
INFO:tensorflow:loss = 0.3282585, step = 8501
INFO:tensorflow:global_step/sec: 11.3992
INFO:tensorflow:loss = 0.46769628, step = 8601 (8.775 sec)
INFO:tensorflow:global_step/sec: 10.8466
INFO:tensorflow:loss = 0.4601719, step = 8701 (9.219 sec)
INFO:tensorflow:global_step/sec: 11.005
INFO:tensorflow:loss = 0.25620446, step = 8801 (9.085 sec)


+ 关于`tf.app.run`有两种情况：

    + 如果你的代码中的入口函数不叫`main()`，而是一个其他名字的函数，如`test()`，则你应该这样写入口`tf.app.run(test())`
    + 如果你的代码中的入口函数叫`main()`，则你就可以把入口写成`tf.app.run()`